Imports utiles

In [ ]:
import os
import cv2
import numpy as np
import torch
import pickle
import json
from PIL import Image
from io import BytesIO
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from playwright.sync_api import sync_playwright


Capture d'écran depuis un lien avec Playwright

In [ ]:
def capture_screenshot(url, save_path, width=1583, height=2048):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page(viewport={"width": width, "height": height})
        page.goto(url, timeout=15000)
        page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
        page.wait_for_timeout(3000)
        page.screenshot(path=save_path, full_page=True)
        browser.close()

# Exécution
url = "https://exemple.com"
img_path = "screenshot.jpg"
capture_screenshot(url, img_path)


Resize de l'image pour la prédiction

In [ ]:
# Résolutions
TARGET_WIDTH = 1024
TARGET_HEIGHT = 1024

# Chargement image
img_original = cv2.imread(img_path)
orig_h, orig_w = img_original.shape[:2]
img_resized = cv2.resize(img_original, (TARGET_WIDTH, TARGET_HEIGHT))
cv2.imwrite("resized_input.jpg", img_resized)

# Sauvegarde des scale ratios
scale_x = orig_w / TARGET_WIDTH
scale_y = orig_h / TARGET_HEIGHT


Chargement du modèle fine-tuné

In [ ]:
# Chargement de la configuration et du modèle
with open("output_detectron_web/config.pkl", "rb") as f:
    cfg = pickle.load(f)

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
predictor = DefaultPredictor(cfg)

# Chargement des metadata
with open("output_detectron_web/metadata.pkl", "rb") as f:
    metadata = pickle.load(f)


Prédiction sur l’image resize

In [ ]:
outputs = predictor(img_resized)
instances = outputs["instances"].to("cpu")


Mise à l’échelle des prédictions vers l’image d’origine

In [ ]:
def scale_boxes(boxes, scale_x, scale_y):
    boxes = boxes.tensor.numpy()
    boxes[:, [0, 2]] *= scale_x  # x1, x2
    boxes[:, [1, 3]] *= scale_y  # y1, y2
    return boxes.astype(int)

scaled_boxes = scale_boxes(instances.pred_boxes, scale_x, scale_y)
classes = instances.pred_classes.numpy()


Annotation de l’image originale avec OpenCV
python
Copier le code


In [ ]:
annotated_img = img_original.copy()
for box, cls in zip(scaled_boxes, classes):
    x1, y1, x2, y2 = box
    label = metadata.thing_classes[cls]
    cv2.rectangle(annotated_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(annotated_img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

cv2.imwrite("annotated_output.jpg", annotated_img)
Image.open("annotated_output.jpg")


Export COCO JSON sur image RESIZE
python
Copier le code


In [ ]:
coco_output = {
    "images": [{"id": 1, "width": TARGET_WIDTH, "height": TARGET_HEIGHT, "file_name": "resized_input.jpg"}],
    "annotations": [],
    "categories": [{"id": i, "name": name} for i, name in enumerate(metadata.thing_classes)]
}

for i, box in enumerate(instances.pred_boxes.tensor.numpy()):
    x, y, x2, y2 = box
    w, h = x2 - x, y2 - y
    coco_output["annotations"].append({
        "id": i,
        "image_id": 1,
        "category_id": int(instances.pred_classes[i]),
        "bbox": [float(x), float(y), float(w), float(h)],
        "area": float(w * h),
        "iscrowd": 0
    })

with open("predictions_resize_coco.json", "w") as f:
    json.dump(coco_output, f, indent=2)

print("✅ Sauvegarde COCO : predictions_resize_coco.json")
